In [1]:
import numpy as np
import xsimlab as xs

In [2]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra

In [3]:
print('xarray-simlab version: ', xs.__version__)
print('phydra version: ', phydra.__version__)

xarray-simlab version:  0.4.1
phydra version:  0+untagged.49.g665b7df.dirty


In [4]:
model = xs.Model({'time':phydra.processes.main.Time, 'grid':phydra.processes.main.Grid0D, 
                  'ibcs':phydra.processes.main.Boundary0D, 'context':phydra.processes.main.GekkoContext, 
                  'comp':phydra.processes.main.Component, 'comp2':phydra.processes.main.Component,
                  'flux':phydra.processes.main.Flux, 'solver':phydra.processes.main.GekkoSolve})

model

<xsimlab.Model (8 processes, 13 inputs)>
time
    days           [in] ('time',) time in days
grid
    length         [in]
    shape          [in]
ibcs
    ibc            [in] Initial Boundary Conditions
context
comp
    dim            [in]
    init           [in]
    label          [in]
comp2
    init           [in]
    label          [in]
    dim            [in]
flux
    c2_label       [in]
    growthrate     [in]
    c1_label       [in]
solver

In [5]:
def phydra_setup(model, input_vars, output_vars):
    return xs.create_setup(model=model, 
                           # necessary for xsimlab
                           clocks={'clock': [0, 1]},
                           input_vars=input_vars,
                           output_vars=output_vars)

in_ds = phydra_setup(
    model=model,
    input_vars={
        'time__days': ('time', np.arange(0, 10)),
        'comp':{'init':1,
                'label':'D',
                'dim':4}, 
        'comp2':{'init':2,
                'label':'X',
                'dim':4}, #the way dims are handled need to be specified in each flux!
        
        'flux':{'growthrate':0.1,
                'c1_label':'D',
                'c2_label':'X'}
    },
    output_vars={}
                 )

In [6]:
with model:
    out_ds = in_ds.xsimlab.run()

[[0. 0. 0. 0.]]
Gekko math dict stores: dict_items([('D', array([[0, 0, 0, 0]], dtype=object))])
[[0. 0. 0. 0.]]
Gekko math dict stores: dict_items([('D', array([[1, 1, 1, 1]], dtype=object)), ('X', array([[0, 0, 0, 0]], dtype=object))])
[[1 1 1 1]]
[[2 2 2 2]]
[[0.0 0.0 0.0 0.0]]
[[0.0 0.0 0.0 0.0]]
[[0 0 0 0]]
[[0 0 0 0]]
step
finalize
[('D', 4), ('X', 4)]
HEEE
D 4
HEEE
X 4
_CountingAttr(counter=228, _default=NOTHING, repr=True, eq=True, order=True, hash=None, init=True, metadata={'var_type': <VarType.VARIABLE: 'variable'>, 'dims': (array([0, 1, 2, 3]),), 'intent': <VarIntent.IN: 'in'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}})
dims: [array([0, 1, 2, 3]), array([0, 1, 2, 3])]
Gekko math dict stores: dict_items([('D', array([[[1.0, 1.1111111111, 1.2345679012, 1.3717421125, 1.5241579028, 1.6935087808, 1.8816764232, 2.0907515813, 2.3230573125, 2.5811747917],
        [1.0, 1.1111111111, 1.2345679012, 1.3717421125, 1.5241579028, 1.6935087808, 1.881676

In [7]:
out_ds



<xarray.Dataset>
Dimensions:           (clock: 2, time: 10)
Coordinates:
  * clock             (clock) int64 0 1
  * time              (time) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    comp2__dim        int64 4
    comp2__init       int64 2
    comp2__label      <U1 'X'
    comp__dim         int64 4
    comp__init        int64 1
    comp__label       <U1 'D'
    flux__c1_label    <U1 'D'
    flux__c2_label    <U1 'X'
    flux__growthrate  float64 0.1
    grid__length      int64 1
    grid__shape       int64 1
    ibcs__ibc         float64 nan
    time__days        (time) int64 0 1 2 3 4 5 6 7 8 9